# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [3]:
view_sentence_range = (137850, 137865)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 137850 to 137865:
new jersey is sometimes chilly during april , but it is usually mild in fall .
california is never nice during october , and it is sometimes rainy in autumn .
our least liked fruit is the apple , but my least liked is the grape .
we plan to visit china in march .
india is sometimes snowy during september , but it is never warm in winter .
france is never busy during march , and it is sometimes pleasant in september .
india is sometimes beautiful during spring , and it is snowy in june .
india is never wet during summer , but it is sometimes chilly in winter .
france is never chilly during january , but it is never mild in october .
the orange is her favorite fruit , but the banana is your favorite .


French sentences 137850 to 137865:
new jersey est parfois froid en avril , mais il est généralement doux à l' au

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [4]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    '''source_id_text = []
    target_id_text = []
    source_text = source_text.split('\n')
    target_text = target_text.split('\n')
    #i = 0
    for sentence in source_text:
        #i = i+1
        id_os = []
        sentence = sentence.split(' ')
        #print(i,sentence)
        for word in sentence:
            if word == '':
                pass
            else:
                id_os.append(source_vocab_to_int[word])
        source_id_text.append(id_os)
    for sentence in target_text:
        sentence = sentence.split(' ')
        id_os = []
        for word in sentence:            
            if word == '':
                pass
            else:
                id_os.append(target_vocab_to_int[word])
        id_os.append(target_vocab_to_int['<EOS>'])
        target_id_text.append(id_os)'''
    
    # 17/09/10 List expression of "for" rouund
    # + [对(x)的操作 for x in 集合 if 条件]或者[对(x,y)的操作 for x in 集合1 for y in 集合2 if 条件]外面必须要有中括号[]
    # 如 [a + b for a in x for b in y if a%2 == 0 and b%2 ==0]
    # 这个操作返回的就是列表。（例子见untitled）
    # + 如果list中的东西是在不断更新的，会自动加入列表。
    # + 还有一个地方很tricky，它在split sentence的时候没有用' ' 空格，而是直接缺省了，
    # str.list() 当不带参数时，默认是以空格作为参数，不管空格在哪，或者有几个 全部会被删去，
    # 但是，如果有参数，那么就会一个一个地删。以i为例，如果两个i连在一起，那么split之后会得到''空字符串
    source_id_text = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_text.split('\n')]
    target_id_text = [[target_vocab_to_int[word] for word in sentence.split()]+[target_vocab_to_int['<EOS>']] for sentence in target_text.split('\n')]
    
    return source_id_text, target_id_text


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [8]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None], 'input')
    target = tf.placeholder(tf.int32, [None, None])
    lr = tf.placeholder(tf.float32, None)
    keep_prob = tf.placeholder(tf.float32, None, 'keep_prob')
    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_length = tf.reduce_max(tf.reduce_max(target_sequence_length), name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    return inputs, target, lr, keep_prob, target_sequence_length, max_target_length, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [9]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    # 3.12.8 这里是用来告诉第二个网络，可以开始产生输出了
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [10]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    inputs = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    cells = []
    for _ in range(num_layers):
        cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer= tf.random_uniform_initializer(-0.1,0.1,seed=2))
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob= keep_prob)
        cells.append(cell)
    rnn = tf.contrib.rnn.MultiRNNCell(cells)
    rnn_output, rnn_state = tf.nn.dynamic_rnn(rnn, inputs, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return rnn, rnn_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=target_sequence_length,
                                                        time_major=False)
    
    # 17/09/10 加入弃权
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, keep_prob)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       encoder_state,
                                                       output_layer) 
    
    training_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                   impute_finished=True,
                                                                   maximum_iterations=max_summary_length)
    
    return training_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [12]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size])
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                start_tokens,
                                                                end_of_sequence_id)
    
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        encoder_state,
                                                        output_layer)
    
    inference_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                    impute_finished=True,
                                                                    maximum_iterations=max_target_sequence_length)
    
    return inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [13]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # Embed the target sequences
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    # Construct the decoder LSTM cell
    cells = []
    for _ in range(num_layers):
        cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer= tf.random_uniform_initializer(-0.1,0.1,seed=2))
        cells.append(cell)
    dec_cell = tf.contrib.rnn.MultiRNNCell(cells)
    
    # Create an output layer
    output_layer = Dense(target_vocab_size, kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    # Training
    # 关于variable问题的详解见：http://wiki.jikexueyuan.com/project/tensorflow-zh/how_tos/variable_scope.html
    # 17/09/10 新的共享变量域的写法：decoding_scope.reuse_variables()
    with tf.variable_scope("decode") as decoding_scope:
        train_logits = decoding_layer_train(encoder_state, dec_cell, dec_embed_input,\
                                            target_sequence_length, max_target_sequence_length, output_layer, keep_prob)

    # Inference
        decoding_scope.reuse_variables()
        infer_logits = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, target_vocab_to_int['<GO>'],\
                                            target_vocab_to_int['<EOS>'],max_target_sequence_length,\
                                            target_vocab_size, output_layer,batch_size, keep_prob)
    
    return train_logits, infer_logits



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Apply embedding to the input data for the encoder.
- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Apply embedding to the target data for the decoder.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [14]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # Apply embedding to the input data for the encoder.
    # 为什么下面encoding_layer里面已经有embed的代码但是上面还是要再运行一次？    
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, enc_embedding_size)
     
    # Encode the input using encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,
    #                                       \source_sequence_length, source_vocab_size, encoding_embedding_size).
    enc_output, enc_state = encoding_layer(input_data, 
                                  rnn_size, 
                                  num_layers, 
                                  keep_prob,
                                  source_sequence_length,
                                  source_vocab_size, 
                                  enc_embedding_size)
    
    # Process target data
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    # Decode the encoded input 
    # decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length,\
    # rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)
    
    #千万注意实参与形参对应，看着看着就混了
    training_decoder_output, inference_decoder_output = decoding_layer(dec_input,
                                                                       enc_state,
                                                                       target_sequence_length,
                                                                       max_target_sentence_length,
                                                                       rnn_size,
                                                                       num_layers,
                                                                       target_vocab_to_int, 
                                                                       target_vocab_size,
                                                                       batch_size,
                                                                       keep_prob,
                                                                       dec_embedding_size) 
    
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [35]:
# Number of Epochs
# 17/09/09 据说epochs还是要5-10比较好，2果然太少了
# 17/09/11 降低了learning rate，提高了echo，这说明？学得更细致了？
epochs = 8
# Batch Size
# 如果可以的话试一下更大的 batch size，为什么？是因为总量太大吗？
batch_size = 512
# RNN Size
# 17/09/09 将rnn_size从128增加到256，让模型能够处理更复杂的任务 
# 好像rnn还可以再大一点，如果出现了“out off memory”的报错，就减小batch的大小
rnn_size = 300
# Number of Layers
# 3层会发生什么？准确率会提高吗？会提高多少？
num_layers = 2
# Embedding Size
# Embedding 用于捕捉词汇的语义（semantic）信息，通常取64,128,196等值。但要注意不能超过总的词汇量（此处是227，见explore the data），
# 否则就变成独热编码了
# 第三次改：64还是太小了，size差不多应该是所有词的一半多一些。
# 更高的validation accuracy不意味着更好的翻译质量，因为 embedding size不够，翻译的时候会倾向于选择比较少的词。（详见
# Because of the limited embedding size, the number of words is reduced for translation.）
encoding_embedding_size = 196
decoding_embedding_size = 196
# Learning Rate
# 17/09/11 从0.01减少为0.005
learning_rate = 0.005
# Dropout Keep Probability
# 17/09/11 从0.7减少为0.5
keep_probability = 0.5
display_step = 2

# (8,512,300,2,196,196,0.005,0.5)
# (7,512,300,2,128,128,0.005,0.5) = 0.99/0.98 但是结果还是 un (black) =。 =....
# (10,256,384,2,128,128,0.005,0.5) 不明白为什么要这样做，慢了好多。
# (6,256,256,2,64,64,0.01,0.7) = 0.98/0.96
# (2,256,256,2,32,32,0.01,0.7) = 0.91

# 目标是两个准确率90%以上

### Build the Graph
Build the graph using the neural network you implemented.

In [36]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [37]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [38]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batc h_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    2/269 - Train Accuracy: 0.2855, Validation Accuracy: 0.3307, Loss: 3.8641
Epoch   0 Batch    4/269 - Train Accuracy: 0.3010, Validation Accuracy: 0.3677, Loss: 3.7078
Epoch   0 Batch    6/269 - Train Accuracy: 0.3305, Validation Accuracy: 0.3589, Loss: 3.2412
Epoch   0 Batch    8/269 - Train Accuracy: 0.3095, Validation Accuracy: 0.3714, Loss: 3.2539
Epoch   0 Batch   10/269 - Train Accuracy: 0.3229, Validation Accuracy: 0.3881, Loss: 3.1150
Epoch   0 Batch   12/269 - Train Accuracy: 0.3070, Validation Accuracy: 0.3689, Loss: 3.0351
Epoch   0 Batch   14/269 - Train Accuracy: 0.3491, Validation Accuracy: 0.3912, Loss: 2.8820
Epoch   0 Batch   16/269 - Train Accuracy: 0.3655, Validation Accuracy: 0.3934, Loss: 2.8097
Epoch   0 Batch   18/269 - Train Accuracy: 0.3532, Validation Accuracy: 0.4156, Loss: 2.8876
Epoch   0 Batch   20/269 - Train Accuracy: 0.3813, Validation Accuracy: 0.4363, Loss: 2.8023
Epoch   0 Batch   22/269 - Train Accuracy: 0.4275, Validation Accuracy

Epoch   0 Batch  180/269 - Train Accuracy: 0.6514, Validation Accuracy: 0.6658, Loss: 0.5461
Epoch   0 Batch  182/269 - Train Accuracy: 0.6606, Validation Accuracy: 0.6648, Loss: 0.5398
Epoch   0 Batch  184/269 - Train Accuracy: 0.6459, Validation Accuracy: 0.6655, Loss: 0.5557
Epoch   0 Batch  186/269 - Train Accuracy: 0.6423, Validation Accuracy: 0.6633, Loss: 0.5490
Epoch   0 Batch  188/269 - Train Accuracy: 0.6726, Validation Accuracy: 0.6729, Loss: 0.5153
Epoch   0 Batch  190/269 - Train Accuracy: 0.6552, Validation Accuracy: 0.6533, Loss: 0.5076
Epoch   0 Batch  192/269 - Train Accuracy: 0.6679, Validation Accuracy: 0.6733, Loss: 0.5171
Epoch   0 Batch  194/269 - Train Accuracy: 0.6674, Validation Accuracy: 0.6649, Loss: 0.5193
Epoch   0 Batch  196/269 - Train Accuracy: 0.6376, Validation Accuracy: 0.6674, Loss: 0.5113
Epoch   0 Batch  198/269 - Train Accuracy: 0.6418, Validation Accuracy: 0.6633, Loss: 0.5327
Epoch   0 Batch  200/269 - Train Accuracy: 0.6736, Validation Accuracy

Epoch   1 Batch   92/269 - Train Accuracy: 0.8889, Validation Accuracy: 0.8829, Loss: 0.1415
Epoch   1 Batch   94/269 - Train Accuracy: 0.8787, Validation Accuracy: 0.8795, Loss: 0.1598
Epoch   1 Batch   96/269 - Train Accuracy: 0.8632, Validation Accuracy: 0.8746, Loss: 0.1492
Epoch   1 Batch   98/269 - Train Accuracy: 0.8810, Validation Accuracy: 0.8798, Loss: 0.1405
Epoch   1 Batch  100/269 - Train Accuracy: 0.9010, Validation Accuracy: 0.8895, Loss: 0.1352
Epoch   1 Batch  102/269 - Train Accuracy: 0.8940, Validation Accuracy: 0.8860, Loss: 0.1292
Epoch   1 Batch  104/269 - Train Accuracy: 0.8901, Validation Accuracy: 0.8927, Loss: 0.1292
Epoch   1 Batch  106/269 - Train Accuracy: 0.8854, Validation Accuracy: 0.8918, Loss: 0.1254
Epoch   1 Batch  108/269 - Train Accuracy: 0.9002, Validation Accuracy: 0.8812, Loss: 0.1258
Epoch   1 Batch  110/269 - Train Accuracy: 0.9062, Validation Accuracy: 0.8844, Loss: 0.1189
Epoch   1 Batch  112/269 - Train Accuracy: 0.9066, Validation Accuracy

Epoch   2 Batch    4/269 - Train Accuracy: 0.9338, Validation Accuracy: 0.9341, Loss: 0.0484
Epoch   2 Batch    6/269 - Train Accuracy: 0.9462, Validation Accuracy: 0.9282, Loss: 0.0438
Epoch   2 Batch    8/269 - Train Accuracy: 0.9497, Validation Accuracy: 0.9401, Loss: 0.0478
Epoch   2 Batch   10/269 - Train Accuracy: 0.9447, Validation Accuracy: 0.9453, Loss: 0.0401
Epoch   2 Batch   12/269 - Train Accuracy: 0.9422, Validation Accuracy: 0.9449, Loss: 0.0508
Epoch   2 Batch   14/269 - Train Accuracy: 0.9374, Validation Accuracy: 0.9341, Loss: 0.0440
Epoch   2 Batch   16/269 - Train Accuracy: 0.9335, Validation Accuracy: 0.9398, Loss: 0.0470
Epoch   2 Batch   18/269 - Train Accuracy: 0.9484, Validation Accuracy: 0.9428, Loss: 0.0476
Epoch   2 Batch   20/269 - Train Accuracy: 0.9434, Validation Accuracy: 0.9313, Loss: 0.0435
Epoch   2 Batch   22/269 - Train Accuracy: 0.9542, Validation Accuracy: 0.9477, Loss: 0.0420
Epoch   2 Batch   24/269 - Train Accuracy: 0.9402, Validation Accuracy

Epoch   2 Batch  182/269 - Train Accuracy: 0.9593, Validation Accuracy: 0.9588, Loss: 0.0296
Epoch   2 Batch  184/269 - Train Accuracy: 0.9618, Validation Accuracy: 0.9546, Loss: 0.0274
Epoch   2 Batch  186/269 - Train Accuracy: 0.9646, Validation Accuracy: 0.9577, Loss: 0.0232
Epoch   2 Batch  188/269 - Train Accuracy: 0.9663, Validation Accuracy: 0.9584, Loss: 0.0253
Epoch   2 Batch  190/269 - Train Accuracy: 0.9594, Validation Accuracy: 0.9635, Loss: 0.0285
Epoch   2 Batch  192/269 - Train Accuracy: 0.9661, Validation Accuracy: 0.9579, Loss: 0.0300
Epoch   2 Batch  194/269 - Train Accuracy: 0.9613, Validation Accuracy: 0.9512, Loss: 0.0295
Epoch   2 Batch  196/269 - Train Accuracy: 0.9510, Validation Accuracy: 0.9497, Loss: 0.0303
Epoch   2 Batch  198/269 - Train Accuracy: 0.9515, Validation Accuracy: 0.9706, Loss: 0.0293
Epoch   2 Batch  200/269 - Train Accuracy: 0.9563, Validation Accuracy: 0.9578, Loss: 0.0253
Epoch   2 Batch  202/269 - Train Accuracy: 0.9617, Validation Accuracy

...

Epoch   4 Batch  184/269 - Train Accuracy: 0.9747, Validation Accuracy: 0.9743, Loss: 0.0154
Epoch   4 Batch  186/269 - Train Accuracy: 0.9797, Validation Accuracy: 0.9659, Loss: 0.0111
Epoch   4 Batch  188/269 - Train Accuracy: 0.9791, Validation Accuracy: 0.9641, Loss: 0.0137
Epoch   4 Batch  190/269 - Train Accuracy: 0.9792, Validation Accuracy: 0.9728, Loss: 0.0163
Epoch   4 Batch  192/269 - Train Accuracy: 0.9860, Validation Accuracy: 0.9684, Loss: 0.0132
Epoch   4 Batch  194/269 - Train Accuracy: 0.9797, Validation Accuracy: 0.9657, Loss: 0.0161
Epoch   4 Batch  196/269 - Train Accuracy: 0.9777, Validation Accuracy: 0.9607, Loss: 0.0153
Epoch   4 Batch  198/269 - Train Accuracy: 0.9809, Validation Accuracy: 0.9610, Loss: 0.0156
Epoch   4 Batch  200/269 - Train Accuracy: 0.9860, Validation Accuracy: 0.9703, Loss: 0.0123
Epoch   4 Batch  202/269 - Train Accuracy: 0.9798, Validation Accuracy: 0.9622, Loss: 0.0161
Epoch   4 Batch  204/269 - Train Accuracy: 0.9813, Validation Accuracy

Epoch   5 Batch   96/269 - Train Accuracy: 0.9818, Validation Accuracy: 0.9754, Loss: 0.0139
Epoch   5 Batch   98/269 - Train Accuracy: 0.9794, Validation Accuracy: 0.9713, Loss: 0.0137
Epoch   5 Batch  100/269 - Train Accuracy: 0.9844, Validation Accuracy: 0.9711, Loss: 0.0125
Epoch   5 Batch  102/269 - Train Accuracy: 0.9807, Validation Accuracy: 0.9756, Loss: 0.0118
Epoch   5 Batch  104/269 - Train Accuracy: 0.9833, Validation Accuracy: 0.9738, Loss: 0.0127
Epoch   5 Batch  106/269 - Train Accuracy: 0.9827, Validation Accuracy: 0.9712, Loss: 0.0105
Epoch   5 Batch  108/269 - Train Accuracy: 0.9827, Validation Accuracy: 0.9685, Loss: 0.0114
Epoch   5 Batch  110/269 - Train Accuracy: 0.9772, Validation Accuracy: 0.9726, Loss: 0.0111
Epoch   5 Batch  112/269 - Train Accuracy: 0.9794, Validation Accuracy: 0.9687, Loss: 0.0146
Epoch   5 Batch  114/269 - Train Accuracy: 0.9834, Validation Accuracy: 0.9708, Loss: 0.0119
Epoch   5 Batch  116/269 - Train Accuracy: 0.9851, Validation Accuracy

Epoch   6 Batch    8/269 - Train Accuracy: 0.9857, Validation Accuracy: 0.9772, Loss: 0.0130
Epoch   6 Batch   10/269 - Train Accuracy: 0.9888, Validation Accuracy: 0.9723, Loss: 0.0106
Epoch   6 Batch   12/269 - Train Accuracy: 0.9768, Validation Accuracy: 0.9750, Loss: 0.0157
Epoch   6 Batch   14/269 - Train Accuracy: 0.9803, Validation Accuracy: 0.9739, Loss: 0.0103
Epoch   6 Batch   16/269 - Train Accuracy: 0.9792, Validation Accuracy: 0.9769, Loss: 0.0132
Epoch   6 Batch   18/269 - Train Accuracy: 0.9805, Validation Accuracy: 0.9680, Loss: 0.0132
Epoch   6 Batch   20/269 - Train Accuracy: 0.9807, Validation Accuracy: 0.9732, Loss: 0.0128
Epoch   6 Batch   22/269 - Train Accuracy: 0.9808, Validation Accuracy: 0.9703, Loss: 0.0119
Epoch   6 Batch   24/269 - Train Accuracy: 0.9797, Validation Accuracy: 0.9784, Loss: 0.0121
Epoch   6 Batch   26/269 - Train Accuracy: 0.9748, Validation Accuracy: 0.9680, Loss: 0.0100
Epoch   6 Batch   28/269 - Train Accuracy: 0.9820, Validation Accuracy

Epoch   6 Batch  186/269 - Train Accuracy: 0.9837, Validation Accuracy: 0.9731, Loss: 0.0095
Epoch   6 Batch  188/269 - Train Accuracy: 0.9869, Validation Accuracy: 0.9735, Loss: 0.0099
Epoch   6 Batch  190/269 - Train Accuracy: 0.9860, Validation Accuracy: 0.9742, Loss: 0.0112
Epoch   6 Batch  192/269 - Train Accuracy: 0.9860, Validation Accuracy: 0.9711, Loss: 0.0104
Epoch   6 Batch  194/269 - Train Accuracy: 0.9867, Validation Accuracy: 0.9772, Loss: 0.0104
Epoch   6 Batch  196/269 - Train Accuracy: 0.9809, Validation Accuracy: 0.9747, Loss: 0.0115
Epoch   6 Batch  198/269 - Train Accuracy: 0.9882, Validation Accuracy: 0.9778, Loss: 0.0111
Epoch   6 Batch  200/269 - Train Accuracy: 0.9923, Validation Accuracy: 0.9759, Loss: 0.0087
Epoch   6 Batch  202/269 - Train Accuracy: 0.9860, Validation Accuracy: 0.9767, Loss: 0.0124
Epoch   6 Batch  204/269 - Train Accuracy: 0.9907, Validation Accuracy: 0.9777, Loss: 0.0082
Epoch   6 Batch  206/269 - Train Accuracy: 0.9782, Validation Accuracy

Epoch   7 Batch   98/269 - Train Accuracy: 0.9777, Validation Accuracy: 0.9778, Loss: 0.0111
Epoch   7 Batch  100/269 - Train Accuracy: 0.9874, Validation Accuracy: 0.9765, Loss: 0.0100
Epoch   7 Batch  102/269 - Train Accuracy: 0.9824, Validation Accuracy: 0.9737, Loss: 0.0089
Epoch   7 Batch  104/269 - Train Accuracy: 0.9861, Validation Accuracy: 0.9775, Loss: 0.0101
Epoch   7 Batch  106/269 - Train Accuracy: 0.9808, Validation Accuracy: 0.9720, Loss: 0.0093
Epoch   7 Batch  108/269 - Train Accuracy: 0.9850, Validation Accuracy: 0.9717, Loss: 0.0101
Epoch   7 Batch  110/269 - Train Accuracy: 0.9849, Validation Accuracy: 0.9743, Loss: 0.0086
Epoch   7 Batch  112/269 - Train Accuracy: 0.9767, Validation Accuracy: 0.9739, Loss: 0.0105
Epoch   7 Batch  114/269 - Train Accuracy: 0.9872, Validation Accuracy: 0.9686, Loss: 0.0080
Epoch   7 Batch  116/269 - Train Accuracy: 0.9892, Validation Accuracy: 0.9783, Loss: 0.0085
Epoch   7 Batch  118/269 - Train Accuracy: 0.9773, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [39]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [40]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [41]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    sentence.lower()
    ids = []
    for word in sentence.split(' '):
        if word in vocab_to_int.keys():
            ids.append(vocab_to_int[word])
        else:
            ids.append(vocab_to_int['<UNK>'])
    return ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [43]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [192, 44, 62, 177, 105, 99, 211]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [304, 289, 97, 167, 216, 11, 352, 1]
  French Words: il a vu un camion rouge . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.